In [1]:
import sqlite3
from sqlite3 import Error

import pymongo
from bson.objectid import ObjectId

In [ ]:
conn = sqlite3.connect(r"C:\Users\krist\OneDrive\Υπολογιστής\smarthome.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM 'profil_xristi';")
profiles = cursor.fetchall()
if not profiles:
    print('create new profile \n')
else:
    for i,j in enumerate(profiles):
        print(i+1,j[0])

prof=input('select profile or press 0 to create new profile:')

if prof=='0':
    username = input('username:')
    kwdikos = input('password:')
    alternative_password = input('alternative password:')
    dimosio = int(input('Public use ?:'))
    polaplwn_xriston = int(input('Multiple users ?:'))
    
    isPrimary = int(input('Is this profile primary:'))
    if isPrimary:
        sql=f''' INSERT INTO proteuon_profil(username_pro)
              VALUES('{username}') '''
        cursor.execute(sql)
    
    else:
        primary_username = input('master_username:')
        
        sql=f''' INSERT INTO deutereuon_profil(username_de)
              VALUES('{username}') '''
        cursor.execute(sql)
    
        sql=f''' INSERT INTO parexei_dikaiwmata(primary_username,secondary_username)
              VALUES('{primary_username}','{username}') '''
        cursor.execute(sql)
    
    sql=f''' INSERT INTO profil_xristi(username,kwdikos,alternative_password,dimosio,pollaplwn_xriston)
          VALUES('{username}','{kwdikos}','{alternative_password}',{dimosio},{polaplwn_xriston}) '''
    cursor.execute(sql)
    current_profile=[username]

else:
    current_profile = profiles[int(prof)-1]
    print(current_profile[0])
    while True:
        if input('password:')==current_profile[1]:
            print('Success')
            break
        else:
            print('Incorrect\n')
            


sql = f'''SELECT secondary_username FROM 'parexei_dikaiwmata' 
      WHERE primary_username='{current_profile[0]}';'''
cursor.execute(sql)
slave_profiles = cursor.fetchall()
if slave_profiles:
    for i,j in enumerate(slave_profiles):
        print(i+1,j[0])
    
    monitored_profile = slave_profiles[int(input('select a profile to monitor:'))-1][0]

    cursor.execute("SELECT * FROM syskeyi")
    appliances = cursor.fetchall()


    while True:
        for i,j in enumerate(appliances):
            sql = f'''SELECT * FROM exei_prosvasi 
            WHERE username_prosvasis='{monitored_profile}' and device_id='{j[0]}';'''
            cursor.execute(sql)
            has_access = cursor.fetchall()
            print(i+1,j[1],j[2],end='')
            if has_access:
                print('-access')
            else:
                print('-no access')


        accessdevice = input('select an appliance to give access to or press 0 to exit:')
        if accessdevice == '0':
            break
        else:
            sql=f''' INSERT INTO exei_prosvasi(username_prosvasis,device_id)
                VALUES('{monitored_profile}','{appliances[int(accessdevice)-1][0]}') '''
            cursor.execute(sql)
            conn.commit()
else:
    print('No child profiles')
conn.commit()
cursor.close()
conn.close()

In [ ]:
conn = sqlite3.connect(r"C:\Users\krist\OneDrive\Υπολογιστής\smarthome.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM 'profil_xristi';")
profiles = cursor.fetchall()
if not profiles:
    print('create new profile \n')
else:
    for i,j in enumerate(profiles):
        print(i+1,j[0])

prof=input('select profile or press 0 to create new profile:')


current_profile = profiles[int(prof)-1]
print(current_profile[0])
while True:
    if input('password:')==current_profile[1]:
        print('Success')
        break
    else:
        print('Incorrect\n')
        
sql = f'''SELECT * FROM 'proteuon_profil' 
      WHERE username_pro='{current_profile[0]}';'''
cursor.execute(sql)
isPrimary = cursor.fetchall()

if isPrimary:
    cursor.execute("SELECT * FROM syskeyi")
    appliances = cursor.fetchall()
    


else:
    sql = f'''select exei_prosvasi.device_id,eidos,dwmatio from exei_prosvasi join syskeyi on syskeyi.device_id=exei_prosvasi.device_id
          and username_prosvasis='{current_profile[0]}';'''
    cursor.execute(sql)
    appliances = cursor.fetchall()
    
while True:   
    for i,j in enumerate(appliances):
        print(i+1,j[1],j[2])

    selection = int(input('select an appliance to use or press 0 to exit:'))
    
    if selection==0:
        break
        
    use_appliance = appliances[selection-1]
    client = pymongo.MongoClient('localhost', 27017)
    db = client['smarthome']


    entoles = db['appliances'].find({'_id':ObjectId(use_appliance[0])})[0]['entoles']

    for i,j in enumerate(entoles.keys()):
        print(i+1,j)

    entoli = entoles[list(entoles.keys())[int(input('select command:'))-1]]
    _id = str(entoli['entolh_id'])

    sql=f''' INSERT INTO entoli(command)
              VALUES('{_id}') '''
    cursor.execute(sql)

    sql=f''' INSERT INTO elegxei(command_id_ele,device_id_ele)
              VALUES(last_insert_rowid(),'{use_appliance[0]}') '''
    cursor.execute(sql)

    sql=f''' INSERT INTO pragmatopoiei(username_pragma,command_id_pragma,date_time)
              VALUES('{current_profile[0]}',last_insert_rowid(),CURRENT_TIMESTAMP) '''
    cursor.execute(sql)
    conn.commit()
    
conn.commit()
cursor.close()
conn.close()